In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta

In [ ]:
raw_data=pd.read_csv("../input/shopee-code-league-20/_DA_Order_Brushing/order_brush_order.csv")

In [ ]:
raw_data

In [ ]:
df=raw_data.copy()

In [ ]:
df.head()

In [ ]:
### analysing about data

In [ ]:
df.info()

In [ ]:
### no missing value in any column

In [ ]:
### check for unique values

In [ ]:
len(df["orderid"].unique())

In [ ]:
len(df["shopid"].unique())

In [ ]:
len(df["userid"].unique())

In [ ]:
type(df["event_time"][0])

In [ ]:
### dropping orderid

In [ ]:
dropped_df=df.drop(["orderid"],axis=1)

In [ ]:
dropped_df.head()

In [ ]:
### changing event_time from object to date_time object

In [ ]:
dates_times=pd.to_datetime(dropped_df["event_time"])

In [ ]:
dates_times.head()

In [ ]:
date_time_df=pd.concat([dropped_df.iloc[:,:-1],dates_times],axis=1)

In [ ]:
date_time_df.head()

In [ ]:
date_time_df.info()

In [ ]:
date_time_df.sort_values(["shopid","event_time"],inplace=True)

In [ ]:
### sorting the dataframe in ascending order based on shopid 
### so that details corresponding to each shopid can be arranged together and also in ascending order of time

In [ ]:
date_time_df.reset_index(drop=True,inplace=True)

In [ ]:
date_time_df

In [ ]:
date_time_df["shopid"].value_counts()

In [ ]:
### function to analyze order brushing

**Approach**

The function takes an argument that is a dataframe of all the data related to a single shopid at a time.
The first event time of that shopid and last is taken into a variable.
A for loop is run on index of the dataframe.
The loop is executed until a hour before the last time. We look for the transactions between that event time and one hour from that and create an another dataframe. We calculate the concentrate rate, if it is equal to or greater than 3, we create a series of unique users and a series of the users who are majorly suspected for order brushing.
We take unique users from that array and add them in a list.
for the last hour we iterate an extra loop after we reach our one hour before end_time. Then we break the loop.
Then we create a dictionary key value pair where key is shopid and values are suspected userid's in desigred string format.
We return this dictionary and update the final dictionary in our main loop from where this function is being called

In [ ]:
def order_brushing_analysis(shopid_df):
    start_time=shopid_df["event_time"].iloc[0]
    end_time=shopid_df["event_time"].iloc[-1]
    order_brushing_dict={}
    suspected_users_str=""
    suspected_users_list=[]
    
    for i in range(shopid_df.shape[0]):
                
        if shopid_df["event_time"].iloc[i]<end_time-timedelta(hours=1):
            one_hour=shopid_df["event_time"].iloc[i]+timedelta(hours=1)
            one_hour_df=shopid_df[shopid_df["event_time"].between(shopid_df["event_time"].iloc[i],one_hour)]
            concentrate_rate=one_hour_df.shape[0]/len(one_hour_df["userid"].unique())
            
            if concentrate_rate>=3:
                unique_users=one_hour_df["userid"].value_counts()
                suspected_users=unique_users[unique_users==unique_users.max()].index
                for users in np.unique(suspected_users):
                    suspected_users_list.append(users)
                    
        else:
            one_hour=shopid_df["event_time"].iloc[i]+timedelta(hours=1)
            one_hour_df=shopid_df[shopid_df["event_time"].between(shopid_df["event_time"].iloc[i],one_hour)]
            concentrate_rate=one_hour_df.shape[0]/len(one_hour_df["userid"].unique())
            
            if concentrate_rate>=3:
                unique_users=one_hour_df["userid"].value_counts()
                suspected_users=unique_users[unique_users==unique_users.max()].index
                for users in np.unique(suspected_users):
                    suspected_users_list.append(users)
                
            break
    if len(suspected_users_list)==0:
        order_brushing_dict[shopid_df["shopid"].iloc[0]]="0"
    else:
        suspected_users_list.sort()
        for users in suspected_users_list:
            suspected_users_str+=str(users)+"&"
        order_brushing_dict[shopid_df["shopid"].iloc[0]]=suspected_users_str[:-1]
                    
    return order_brushing_dict
            
                

In [ ]:
final_dict={}

In [ ]:
for shoperid in date_time_df["shopid"].unique():
    new_dict=order_brushing_analysis(date_time_df[date_time_df["shopid"]==shoperid])
    final_dict.update(new_dict)

In [ ]:
final_df=pd.DataFrame()
final_df["shopid"]=list(final_dict.keys())
final_df["userid"]=list(final_dict.values())

In [ ]:
final_df

In [ ]:
### This is my first submission to any kaggle task.
### Please suggest if any improvement or optimization is required.
### Please upvote